In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_moons
import sklearn as skl

2023-05-24 16:59:16.386446: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Study data files
diabetes_binary_health_path = "Ressources/diabetes_binary_health_indicators_BRFSS2015.csv"
# Read the  data and the study results
df_diabetes = pd.read_csv(diabetes_binary_health_path)

df_diabetes.head()


,Diabetes,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0,1,1,1,40,1,0,0,0,0,...,1,0,5,18,15,1,0,9,4,3
1,0,0,0,0,25,1,0,0,1,0,...,0,1,3,0,0,0,0,7,6,1
2,0,1,1,1,28,0,0,0,0,1,...,1,1,5,30,30,1,0,9,4,8
3,0,1,0,1,27,0,0,0,1,1,...,1,0,2,0,0,0,0,11,3,6
4,0,1,1,1,24,0,0,0,1,1,...,1,0,2,3,0,0,0,11,5,4


In [3]:
values = df_diabetes['Diabetes'].value_counts()
values

0    218334
1     35346
Name: Diabetes, dtype: int64

In [4]:
# Convert categorical data to numeric with `pd.get_dummies`
df_diabetes = pd.get_dummies(df_diabetes, dtype=float)

In [5]:
# split the dataset into feature and target
X = df_diabetes.copy()
X.drop('Diabetes', axis=1)
X.head()

,Diabetes,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0,1,1,1,40,1,0,0,0,0,...,1,0,5,18,15,1,0,9,4,3
1,0,0,0,0,25,1,0,0,1,0,...,0,1,3,0,0,0,0,7,6,1
2,0,1,1,1,28,0,0,0,0,1,...,1,1,5,30,30,1,0,9,4,8
3,0,1,0,1,27,0,0,0,1,1,...,1,0,2,0,0,0,0,11,3,6
4,0,1,1,1,24,0,0,0,1,1,...,1,0,2,3,0,0,0,11,5,4


In [6]:
y = df_diabetes['Diabetes'].values.reshape(-1,1)
y[:5]

array([[0],
       [0],
       [0],
       [0],
       [0]])

In [7]:
# split the data into training and testing dadaset by train_test_split.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [8]:
# Create the StandardScaler instance
X_scaler = skl.preprocessing.StandardScaler()

In [9]:
# Fit the Standard Scaler with the training data
X_scaler.fit(X_train)

StandardScaler()

In [10]:
# transform the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [11]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    
    # create model
    model = tf.keras.models.Sequential()
    
    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu', 'tanh', 'sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    model.add(tf.keras.layers.Dense(units=hp.Int('first_units', min_value=1, max_value=12, step=4), activation=activation, input_dim=len(X_train_scaled[0])))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 5)):
        model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i), min_value=1, max_value=12, step=4), activation=activation))
        
    model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [12]:
# Import the kerastuner library
import keras_tuner as kt

# Create a `Hyperband()` tuner instance
tuner = kt.Hyperband(create_model, objective='val_accuracy', max_epochs=50, hyperband_iterations=2)


INFO:tensorflow:Reloading Tuner from ./untitled_project/tuner0.json


In [ ]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled, y_train, epochs=50, validation_data=(X_test_scaled, y_test))

Trial 70 Complete [00h 09m 41s]
val_accuracy: 0.8618259429931641

Best val_accuracy So Far: 1.0
Total elapsed time: 01h 03m 19s

Search: Running Trial #71

Value             |Best Value So Far |Hyperparameter
relu              |tanh              |activation
1                 |9                 |first_units
3                 |3                 |num_layers
1                 |5                 |units_0
1                 |5                 |units_1
9                 |9                 |units_2
5                 |9                 |units_3
1                 |1                 |units_4
17                |6                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
1                 |2                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/17
5946/5946 [==============================] - 55s 9ms/step - loss: 0.1925 - accuracy: 0.8911 - val_loss: 0.1574 - val_accuracy: 0.9053
Epoch 2/17
5946/5946 [==============================] - 

In [ ]:
# Get top 3 model hyperparameters and print the values
top_hyper = tuner.get_best_hyperparameters(3)
for param in top_hyper:
    print(param.values)

In [ ]:
# Evaluate the top 3 models against the test dataset
top_models = tuner.get_best_models(3)
for model in top_models:
    loss, accuracy = model.evaluate(X_test_scaled, y_test, verbose=2)
    print(f"Loss: {loss}, Accuracy: {accuracy}")

In [ ]:
# Compare the performance to the second-best model
second_best_model = tuner.get_best_models(2)[1]
loss, accuracy = second_best_model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {loss}, Accuracy: {accuracy}")

In [19]:
# Creating dummy nonlinear data
X_moons, y_moons = make_moons(n_samples=1000, noise=0.08, random_state=78)
# Transforming y_moons to a vertical vector
y_moons = y_moons.reshape(-1, 1)
# Creating a DataFrame to plot the nonlinear dummy data
df_moons = pd.DataFrame(X_moons, columns=["Feature 1", "Feature 2"])
df_moons["Target"] = y_moons
# Use sklearn to split dataset
X_train, X_test, y_train, y_test = train_test_split(X_moons, y_moons, random_state=78)

# Create scaler instance
X_scaler = skl.preprocessing.StandardScaler()

# Fit the scaler
X_scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [20]:
# Create a Keras Sequential model and add more than one Dense hidden layer
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Dense(units=6, activation='relu', input_dim=2))
model.add(tf.keras.layers.Dense(units=6, activation='relu'))
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the Sequential model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 18        
                                                                 
 dense_1 (Dense)             (None, 6)                 42        
                                                                 
 dense_2 (Dense)             (None, 1)                 7         
                                                                 
Total params: 67
Trainable params: 67
Non-trainable params: 0
_________________________________________________________________


In [21]:
# Compile the model and train over more than 100 epochs
model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])
fit_model = model.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200
24/24 [==============================] - 1s 2ms/step - loss: 0.6014 - accuracy: 0.7653
Epoch 2/200
24/24 [==============================] - 0s 2ms/step - loss: 0.5624 - accuracy: 0.8173
Epoch 3/200
24/24 [==============================] - 0s 3ms/step - loss: 0.5266 - accuracy: 0.8200
Epoch 4/200
24/24 [==============================] - 0s 2ms/step - loss: 0.4911 - accuracy: 0.8293
Epoch 5/200
24/24 [==============================] - 0s 2ms/step - loss: 0.4572 - accuracy: 0.8400
Epoch 6/200
24/24 [==============================] - 0s 2ms/step - loss: 0.4244 - accuracy: 0.8587
Epoch 7/200
24/24 [==============================] - 0s 3ms/step - loss: 0.3957 - accuracy: 0.8760
Epoch 8/200
24/24 [==============================] - 0s 3ms/step - loss: 0.3695 - accuracy: 0.8787
Epoch 9/200
24/24 [==============================] - 0s 3ms/step - loss: 0.3470 - accuracy: 0.8800
Epoch 10/200
24/24 [==============================] - 0s 3ms/step - loss: 0.3279 - accuracy: 0.8880
Epoch 11/

24/24 [==============================] - 0s 3ms/step - loss: 0.0302 - accuracy: 0.9973
Epoch 84/200
24/24 [==============================] - 0s 3ms/step - loss: 0.0291 - accuracy: 0.9973
Epoch 85/200
24/24 [==============================] - 0s 3ms/step - loss: 0.0281 - accuracy: 0.9973
Epoch 86/200
24/24 [==============================] - 0s 3ms/step - loss: 0.0271 - accuracy: 0.9973
Epoch 87/200
24/24 [==============================] - 0s 3ms/step - loss: 0.0263 - accuracy: 0.9973
Epoch 88/200
24/24 [==============================] - 0s 2ms/step - loss: 0.0254 - accuracy: 0.9973
Epoch 89/200
24/24 [==============================] - 0s 2ms/step - loss: 0.0245 - accuracy: 0.9973
Epoch 90/200
24/24 [==============================] - 0s 3ms/step - loss: 0.0238 - accuracy: 0.9973
Epoch 91/200
24/24 [==============================] - 0s 3ms/step - loss: 0.0230 - accuracy: 0.9973
Epoch 92/200
24/24 [==============================] - 0s 3ms/step - loss: 0.0223 - accuracy: 0.9973
Epoch 93/200


In [22]:
# Evaluate the performance of model using the loss and predictive accuracy of the model on the test dataset.
loss, accuracy = model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {loss}, Accuracy: {accuracy}")

8/8 - 0s - loss: 0.0021 - accuracy: 1.0000 - 163ms/epoch - 20ms/step
Loss: 0.0020710655953735113, Accuracy: 1.0
